# POMDP's for Personalized Mammography Screening 

Alexander Goldberg, Lydia Goldberg, and Gabriela Merz 

For our final project, we implemented, solved, and optimized a Partially Observable Markov Decision Process Model for personalized mammography screening. This jupyter notebook describes our project in a clear and concise manner, and allows the reader to play with our implementation and experiment with our results. The notebook is organized as follows: 

1. Model Overview and Implementation 
2. Solving the Model: Monohan's Enumeration Algorithm 
3. Optimization: Eagle's Reduction and AMPLS for LP Solving 
4. Mammogram Decisions

# 1. Model Overview and Implementation 

The model was taken from Ayer, et. al "A POMDP approach for Personalized Mammography Screening" and is pictured below: 

![pomdp](pomdp.png)


We will go in to some aspects of our model which are important for understanding our implementation, however, for a full understanding of our model we refer the reader to our official write-up. The aspects of our model which we will deatil here are: 

1. The States 
2. The Transition Probabilities Between States 
3. The Observation Probabilities 
4. The Current Belief State 



1. **The States**

    a. Death, In situ post-cancer treatment, and invasive post-cancer treatment are fully observable and absorbing states. This means that we know with probability one whether you are in one of these states and once you have entered one of these states, the model ends. We can assume we know with probability one whether you enter in situ post-cancer treatment or invasive post-cancer treatment because of the incredibly accuracy of medical biopsys, and in our model, if you recieve a positive mammogram, you immediately get a biopsy. 

    b. At each state, you can take one of two actions: you can decide to get a mammogram, or you can decide to wait. If you decide to get a mammogram, you will either "observe" a positive or negative result, and if you decide to wait, you will either "observe" cancer on your own (self-detect) or not. Time is incremented in periods of six months, which is consistent with both how often patients get check-ups and the current guidelines for when to get a mammogram

2. **The Transition Probabilities Between States**

    a. The transition probabilities are held constant throughout our model. They were estimated using SEER data, and are detailed in the table below. States 1, 2, 3, 4, 5, 6 correspond to the labeled states in the diagram of our model above. 

| | 0 | 1 | 2 | 3 | 4 | 5 | 
|-|-- |-- |-- |-- |-- |-- | 
|0| .993 |.001 | .002 | 0 | 0 |.004|
|1| 0    |.416 |  .58 | 0 | 0 |.004|
|2| 0    | 0   |  .9  | 0 | 0 | .1 |
|3| 0    | 0   |  0   | 1 | 0 | 0  |
|4| 0    | 0   |  0   | 0 | 1 | 0  |
|5| 0    | 0   |  0   | 0 | 0 | 1  |


    
b. These nubmers are also included in the file stats.py and can also be explained intuitively. If you are healthy, your probability of transitioning to any of the cancer states or death is based off of the population probabilities. If you have any form of cancer, you cannot transition back to healthy, and if you have invasive cancer, you cannot transition back to in-situ cancer. However, you can transition from in-situ to invasive cancer. Once your state is invasive cancer, your probability of death also increases. Because in-situ cancer rarely if ever causes death, we have made the transition probability to death the same as for when you are healthy. 

3. **The Observation Probabilities** 

    a. The observation probabilities (of observing a positive mammogram vs. negative mammogram, a positive self detection vs. a negative self detection) are also given in the file stats.py. These probabilities are based off of the known *sensitivity* and *specificty* of mammograms and self detection. Sensitivity is the likelihood that you observe a positive mammogram given that you have some form of cancer, and specificity is the likelihood that you observe a negative mammogram given that you don't have some form of cancer. 
    
4. **Lump-Sum Rewards**


 




## 1.1 Model Implementation 

In the file model.py you will find our implementation of the model! We have implemented hte POMDP model as a class that takes in the initial belief state -- the probabilities you are healthy, have in situ, or have invasive cancer --, the initial timestep and the last timestep for our model. Because time steps are in incrememnts of six months, and we initialize our model for women of age 40 in order to be consistent with current mammography guidelines, we default from initial time 0 to final time 120, which is consistent with our patient living 40 years. 

# 2. Solving the POMDP: Monohan's Enumeration Algorithm 